#**Accessing the "State of the Art Generative models" and generating respones to queries, Detection of AI Generated Text by Fine Tuning LLM(BERT)**

In this project I have explained how to access LLM "Gemini",by Google with the use of API, and I have also shown the process to fine tune a LLM i.e., "BERT" in here, for the purpose of **"Detection of AI Generated Text"**.

To access State of the Art LLM model "Gemini" we have two methods

1.   Use it on https://gemini.google.com/
2.   If you are a developer, you can use the API key to access it.

The procedure to access Gemini by API is given below:

1.   Visit https://aistudio.google.com/app/apikey and click on Sign up(assuming you have a Gmail account).
2.   On the top you can click on "Create API Key" and use it!





In [1]:
!pip install transformers transformers[torch] tqdm kaggle datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 879.8 kB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import torch
import tqdm
import transformers
from datasets import *
import evaluate
from transformers import AutoTokenizer
from torch.utils.data import DataLoader

2024-03-01 07:42:58.474291: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-01 07:42:58.474436: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-01 07:42:58.743549: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
train_data = pd.read_csv("/kaggle/input/daigt-v4-train-dataset/train_v4_drcat_01.csv")
train_data


,text,label,prompt_name,source,RDizzl3_seven,model
0,Phones\n\nModern humans today are always on th...,0,Phones and driving,persuade_corpus,False,human
1,This essay will explain if drivers should or s...,0,Phones and driving,persuade_corpus,False,human
2,Driving while the use of cellular devices\n\nT...,0,Phones and driving,persuade_corpus,False,human
3,Phones & Driving\n\nDrivers should not be able...,0,Phones and driving,persuade_corpus,False,human
4,Cell Phone Operation While Driving\n\nThe abil...,0,Phones and driving,persuade_corpus,False,human
...,...,...,...,...,...,...
73568,I am writing you today to disagree with your t...,1,Grades for extracurricular activities,persuade_finetuned_llamas,False,llama
73569,"Dear Principal,\n\nIn conclusion, I would obse...",1,Grades for extracurricular activities,persuade_finetuned_llamas,False,llama
73570,"Dear Mrs. Principal,\n\nin these kinds of cons...",1,Grades for extracurricular activities,persuade_finetuned_llamas,False,llama
73571,I enjoyed Form five and excitedly ex claims ed...,1,Grades for extracurricular activities,persuade_finetuned_llamas,False,llama


In [4]:
copy_data = train_data
copy_data

,text,label,prompt_name,source,RDizzl3_seven,model
0,Phones\n\nModern humans today are always on th...,0,Phones and driving,persuade_corpus,False,human
1,This essay will explain if drivers should or s...,0,Phones and driving,persuade_corpus,False,human
2,Driving while the use of cellular devices\n\nT...,0,Phones and driving,persuade_corpus,False,human
3,Phones & Driving\n\nDrivers should not be able...,0,Phones and driving,persuade_corpus,False,human
4,Cell Phone Operation While Driving\n\nThe abil...,0,Phones and driving,persuade_corpus,False,human
...,...,...,...,...,...,...
73568,I am writing you today to disagree with your t...,1,Grades for extracurricular activities,persuade_finetuned_llamas,False,llama
73569,"Dear Principal,\n\nIn conclusion, I would obse...",1,Grades for extracurricular activities,persuade_finetuned_llamas,False,llama
73570,"Dear Mrs. Principal,\n\nin these kinds of cons...",1,Grades for extracurricular activities,persuade_finetuned_llamas,False,llama
73571,I enjoyed Form five and excitedly ex claims ed...,1,Grades for extracurricular activities,persuade_finetuned_llamas,False,llama


In [5]:
print(train_data["model"].unique())
print((train_data["model"].to_numpy() == "human").sum())
print((train_data["model"].to_numpy() != "human").sum())

['human' 'mistral' 'llama' 'gpt' 'claude' 'falcon' 'palm' 'cohere' 'ada'
 'babbage' 'curie' 'davinci']
27373
46200


In [6]:
# Separate the 'Human' class
human_class = train_data[train_data['model'] == 'human']

# Separate the other classes
other_classes = train_data[train_data['model'] != 'human']

# Randomly sample 27373 rows from other classes
balanced_other_classes = other_classes.sample(n=len(human_class), random_state=42)

# Concatenate the 'Human' class and the balanced other classes
balanced_df = pd.concat([human_class, balanced_other_classes])

# Shuffle the dataframe
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Now 'balanced_df' contains the balanced dataset
balanced_df.drop(["source","RDizzl3_seven","model","prompt_name"],axis = 1, inplace=True)

In [7]:
balanced_df

,text,label
0,What do you think made the face on the side of...,0
1,I am strongly against this policy that the pri...,0
2,"Hi, I'm Kelvin and I go against this technolog...",0
3,Many people have come to believe that there is...,0
4,A life without cars sure sounds like a great i...,0
...,...,...
54741,"Have you ever dreamed of adventure, traveling ...",1
54742,"Important lesson,can tech you lessons, about ...",1
54743,Title: A Reflection on the Progress and Benefi...,1
54744,Texting & Driving\n\nUsing your phone while dr...,0


In [8]:
balanced_dataset = Dataset.from_pandas(balanced_df)
balanced_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 54746
})

In [9]:
data = balanced_dataset.train_test_split(train_size=0.8, shuffle=True)
dataset = DatasetDict({
    'train': data['train'],
    'val': data['test']})
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 43796
    })
    val: Dataset({
        features: ['text', 'label'],
        num_rows: 10950
    })
})

In [10]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding=True, max_length=512)

In [11]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_dataset = dataset.map(function=tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

  0%|          | 0/44 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

In [12]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 43796
    })
    val: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 10950
    })
})

In [13]:
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")

In [14]:
train_loader = DataLoader(tokenized_dataset['train'], batch_size=16, shuffle=True)
eval_loader = DataLoader(tokenized_dataset['val'], batch_size=16,shuffle=True)

In [15]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print('cuda' if torch.cuda.is_available() else 'cpu')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


cuda


In [16]:
metric = evaluate.load("accuracy")

In [17]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [18]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [19]:
from transformers import TrainingArguments,Trainer
training_args = TrainingArguments(output_dir="/kaggle/working/run3", per_device_train_batch_size = 16, save_total_limit=2, disable_tqdm=False, load_best_model_at_end=True, num_train_epochs=5, evaluation_strategy="steps")
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['val'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [20]:
import wandb

# Disable wandb logging
wandb.init(mode="disabled")


In [21]:
trainer.train(resume_from_checkpoint='/kaggle/input/outputftllm/results/run3/checkpoint-6500')

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss


Could not locate the best model at run3/checkpoint-1500/pytorch_model.bin, if you are running a distributed training on multiple nodes, you should activate `--save_on_each_node`.


TrainOutput(global_step=6845, training_loss=0.0008550581224997432, metrics={'train_runtime': 556.6693, 'train_samples_per_second': 393.375, 'train_steps_per_second': 12.296, 'total_flos': 5.76160589027328e+16, 'train_loss': 0.0008550581224997432, 'epoch': 5.0})